In [ ]:
import sys
# setting path
sys.path.append('../')

import os
import glob

import matplotlib.pyplot as plt
import numpy as np

from utils.datasets import WSSBDatasetTest
from utils.utils_BCD import od2rgb_np, C_to_OD_np, C_to_RGB_np

from utils.utils_data import get_camelyon_test_dataloader, get_wssb_test_dataloader

In [22]:
import torch
import torchvision
import torchvision.models.resnet as resnet

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [34]:
model = torchvision.models.mobilenet_v3_small(pretrained=False)
#model = torchvision.models.efficientnet_b0(pretrained=False)
#model = torchvision.models.vgg16(pretrained=False)

In [32]:
count_parameters(model)

2542856

In [35]:
print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [ ]:
model.classifier = torch.nn.Linear(576, 2)

In [2]:
out_Mnet_mean = torch.rand(32, 3, 2)
out_Mnet_var = torch.rand(32, 1, 2)
out_Cnet = torch.rand(32, 2, 224, 224)
Cflat = out_Cnet.view(-1, 2, 224 * 224) #shape: (batch, 2, patch_size * patch_size)


In [ ]:
torch.rand_like(out_Mnet_mean)*out_Mnet_var, Cflat

In [ ]:
sample_M = out_Mnet_mean + torch.rand_like(out_Mnet_mean)*out_MNet_var

In [ ]:
prueba = WSSBDatasetTest("/data/BasesDeDatos/Alsubaie/Data/")

In [ ]:
img, od_img, mR, C_gt, M_gt = prueba[0]

In [ ]:
print(C_gt.min(), C_gt.max())

In [ ]:
plt.subplot(121)
plt.imshow(C_gt[0],cmap='gray')
plt.subplot(122)
plt.imshow(C_gt[1],cmap='gray')
plt.show()

In [ ]:
M_gt.shape

In [ ]:
C_rgb = C_to_RGB_np(C_gt, M_gt)
C_rgb = np.clip(C_rgb, 0, 255).astype(np.uint8)
plt.subplot(121)
plt.imshow(C_rgb[0, :, :, :])
plt.subplot(122)
plt.imshow(C_rgb[1, :, :, :])
plt.show()

In [ ]:
C_gt_flat = C_gt.squeeze().reshape(2, -1)
Y_rec = np.dot(M_gt.squeeze(), C_gt_flat)
print(Y_rec.min(), Y_rec.max())
Y_rec = od2rgb_np(Y_rec)
print(Y_rec.min(), Y_rec.max())
Y_rec = np.clip(Y_rec, 0, 255).astype(np.uint8)
Y_rec = Y_rec.reshape(3, 2000, 2000)
Y_rec = Y_rec.transpose(1,2,0)

In [ ]:
plt.imshow(Y_rec)

In [ ]:
M_gt = M_gt.reshape(1, 3, 2)
C_gt = C_gt.reshape(1, 2, 2000, 2000)
print(M_gt.shape, C_gt.shape)

In [ ]:
A = np.einsum('bcs, bswh -> bschw', M_gt, C_gt)
A = np.clip(od2rgb_np(A), 0, 255).astype(np.uint8)
A.shape

In [ ]:
plt.subplot(121)
plt.imshow(A[0, 0, :, :, :].transpose(1, 2, 0))
plt.subplot(122)
plt.imshow(A[0, 1, :, :, :].transpose(1, 2, 0))
plt.show()